In [ ]:
!pip install wordcloud         # 기본
# conda 환경을 쓰신다면 ↘
# !conda install -c conda-forge wordcloud -y


In [ ]:
"""
🍖 네이버 API 활용 곱창집 트렌드 분석 시스템
-------------------------------------------------
* Python 3.9+
* requests, pandas, numpy, matplotlib, seaborn, wordcloud
* 안전한 API 키 관리를 위해 `.env`, 환경 변수, Secret Manager 등을 권장합니다.
  (여기서는 사용자 요청에 따라 키를 직접 파라미터로 전달하도록 예시)
"""

# ========================
# 1. 기본 라이브러리 import
# ========================
from __future__ import annotations

import os
import time
import re
import json
from collections import Counter
from datetime import datetime
from typing import Dict, List, Optional

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud

plt.rcParams["font.family"] = "AppleGothic"  # macOS 기준, Windows = "Malgun Gothic"

# ========================
# 2. 네이버 검색 API 클라이언트
# ========================
class NaverAPIClient:
    """네이버 검색 API 래퍼 클래스

    Parameters
    ----------
    client_id : str
        네이버 개발자센터 Client ID
    client_secret : str
        네이버 개발자센터 Client Secret
    rate_limit : float, optional
        호출 간 최소 대기 시간(초). 기본 0.2 초.
    """

    BASE_URL = "https://openapi.naver.com/v1/search"

    def __init__(self, client_id: str, client_secret: str, *, rate_limit: float = 0.2):
        self.client_id = client_id
        self.client_secret = client_secret
        self.rate_limit = rate_limit
        self.headers = {
            "X-Naver-Client-Id": self.client_id,
            "X-Naver-Client-Secret": self.client_secret,
        }

    # ------------------------
    # 내부 헬퍼: 요청 & 예외 처리
    # ------------------------
    def _request(self, endpoint: str, params: Dict[str, str]) -> Dict:
        url = f"{self.BASE_URL}/{endpoint}.json"
        resp = requests.get(url, headers=self.headers, params=params, timeout=10)
        if resp.status_code != 200:
            raise RuntimeError(
                f"Naver API error {resp.status_code}: {resp.text[:200]}…"
            )
        time.sleep(self.rate_limit)
        return resp.json()

    # ------------------------
    # 퍼블릭: 뉴스·블로그·카페 검색
    # ------------------------
    def search(self, query: str, *, source: str, display: int = 100, max_pages: int = 1) -> List[Dict]:
        """지정 source(news/blog/cafearticle)에서 최대 `max_pages` × `display` 건 수집"""
        items: List[Dict] = []
        for page in range(max_pages):
            start = page * display + 1
            payload = {
                "query": query,
                "display": display,
                "start": start,
                "sort": "date",  # 최신순
            }
            data = self._request(source, payload)
            items.extend(data.get("items", []))
            if len(data.get("items", [])) < display:
                break  # 마지막 페이지 도달
        return items

    # 편의 메서드
    def search_news(self, query: str, **kw):
        return self.search(query, source="news", **kw)

    def search_blog(self, query: str, **kw):
        return self.search(query, source="blog", **kw)

    def search_cafe(self, query: str, **kw):
        return self.search(query, source="cafearticle", **kw)


# ========================
# 3. 곱창 트렌드 분석기
# ========================
class GopchangTrendAnalyzer:
    """곱창 키워드 & 지역 기반 트렌드 크롤·분석"""

    BASE_KEYWORDS = [
        "곱창",
        "막창",
        "대창",
        "양곱창",
        "곱창전골",
        "곱창구이",
        "한우곱창",
    ]

    TOP_DISTRICTS = [
        "강남역",
        "종로",
        "명동",
        "신촌",
        "홍대",
        "노량진",
        "노원",
        "잠실",
        "여의도",
        "연남동",
    ]

    POSITIVE = [
        "맛있",
        "좋",
        "훌륭",
        "최고",
        "대박",
        "추천",
        "만족",
        "신선",
        "부드러",
        "쫄깃",
        "고소",
        "달콤",
        "감동",
    ]
    NEGATIVE = [
        "맛없",
        "별로",
        "실망",
        "최악",
        "비추천",
        "불만족",
        "질겨",
        "싱거",
        "짜",
        "비싸",
        "불친절",
        "더러",
    ]

    def __init__(self, client: NaverAPIClient):
        self.client = client

    # ---------------------
    # 데이터 수집
    # ---------------------
    def collect(self, days: int = 30) -> Dict[str, List[Dict]]:
        """BASE_KEYWORDS + 지역 조합 키워드로 뉴스·블로그·카페 데이터 수집"""
        print("🔍 Collecting trend data…")
        cutoff_date = datetime.now() - pd.Timedelta(days=days)

        data = {"news": [], "blog": [], "cafe": []}

        # 1) 기본 키워드 3개만 (API 쿼터 고려)
        for kw in self.BASE_KEYWORDS[:3]:
            data["news"].extend(self.client.search_news(kw, display=100, max_pages=1))
            data["blog"].extend(self.client.search_blog(kw, display=100, max_pages=1))

        # 2) "지역+키워드" 상위 10개 조합만 블로그 수집
        combo_keywords = [f"{d} {k}" for d in self.TOP_DISTRICTS[:5] for k in self.BASE_KEYWORDS[:2]]
        for kw in combo_keywords:
            data["blog"].extend(self.client.search_blog(kw, display=50, max_pages=1))

        # 3) 날짜 필터링 & 정규화
        for src in data:
            data[src] = [item for item in data[src] if self._is_recent(item, cutoff_date)]

        print(
            f"✅ Fetched: News={len(data['news'])}, Blog={len(data['blog'])}, Cafe={len(data['cafe'])}"
        )
        return data

    @staticmethod
    def _is_recent(item: Dict, cutoff: datetime) -> bool:
        try:
            pub_date = datetime.strptime(item.get("pubDate", ""), "%a, %d %b %Y %H:%M:%S %z")
            return pub_date.replace(tzinfo=None) >= cutoff
        except Exception:
            return True  # 날짜 파싱 실패 시 포함

    # ---------------------
    # 전처리 & 감정 분석
    # ---------------------
    def preprocess(self, raw: Dict[str, List[Dict]]) -> pd.DataFrame:
        rows: List[Dict] = []
        for src, items in raw.items():
            for itm in items:
                text = f"{itm.get('title', '')} {itm.get('description', '')}"
                rows.append(
                    {
                        "source": src,
                        "title": itm.get("title", ""),
                        "description": itm.get("description", ""),
                        "link": itm.get("link", ""),
                        "pub_date": itm.get("pubDate", ""),
                        "sentiment": self._sentiment(text),
                        "districts": self._extract_districts(text),
                    }
                )
        return pd.DataFrame(rows)

    def _sentiment(self, txt: str) -> str:
        txt_clean = re.sub(r"<[^>]+>", "", txt)
        pos = sum(word in txt_clean for word in self.POSITIVE)
        neg = sum(word in txt_clean for word in self.NEGATIVE)
        if pos > neg:
            return "positive"
        if neg > pos:
            return "negative"
        return "neutral"

    def _extract_districts(self, txt: str) -> List[str]:
        txt_clean = re.sub(r"<[^>]+>", "", txt)
        return [d for d in self.TOP_DISTRICTS if d in txt_clean]

    # ---------------------
    # 상권별 인사이트
    # ---------------------
    def district_insight(self, df: pd.DataFrame, district: str) -> str:
        subset = df[df["districts"].apply(lambda lst: district in lst if isinstance(lst, list) else False)]
        if subset.empty:
            return f"❌ {district} 언급 없음"

        sentiment_ratio = (
            subset["sentiment"].value_counts(normalize=True).get("positive", 0.0)
        )
        keywords = re.findall(r"[가-힣A-Za-z]+창", " ".join(subset["title"] + " " + subset["description"]))
        hot_kw = ", ".join([k for k, _ in Counter(keywords).most_common(3)])
        main_src = subset["source"].value_counts().idxmax()

        return (
            f"📍 {district} 트렌드\n"
            f"— 총 언급: {len(subset)}건\n"
            f"— 긍정 비율: {sentiment_ratio:.0%}\n"
            f"— 인기 키워드: {hot_kw or '-'}\n"
            f"— 주요 소스: {main_src}"
        )

# ========================
# 4. 타겟 고객 세그먼트 분석기
# ========================
class GopchangTargetAnalyzer:
    SEGMENTS = {
        "직장인 회식족": {
            "특징": ["회식", "야식", "회사", "동료", "술"],
            "시간": "18:00–22:00",
            "메뉴": ["곱창구이", "막창", "소주"],
            "민감도": "중간",
        },
        "젊은층 데이트족": {
            "특징": ["데이트", "연인", "분위기", "인스타", "맛집"],
            "시간": "19:00–23:00",
            "메뉴": ["곱창전골", "양곱창", "와인"],
            "민감도": "낮음",
        },
        "가족 외식족": {
            "특징": ["가족", "아이", "주말", "넓은", "점심"],
            "시간": "12:00–14:00 / 18:00–20:00",
            "메뉴": ["곱창전골", "한우곱창", "밥류"],
            "민감도": "높음",
        },
        "관광객": {
            "특징": ["관광", "여행", "한국", "전통", "체험"],
            "시간": "12:00–15:00 / 18:00–21:00",
            "메뉴": ["전통 곱창전골", "한국술"],
            "민감도": "낮음",
        },
    }

    def strategy(self, district: str, district_type: str) -> str:
        if district_type == "프리미엄 비즈니스":
            pri, sec = "직장인 회식족", "젊은층 데이트족"
        elif district_type == "대학가":
            pri, sec = "젊은층 데이트족", "직장인 회식족"
        elif district_type == "관광/전통상권":
            pri, sec = "관광객", "가족 외식족"
        else:
            pri, sec = "가족 외식족", "직장인 회식족"

        def fmt(seg: str) -> str:
            s = self.SEGMENTS[seg]
            return (
                f"● {seg}\n"
                f"  ├ 특징: {', '.join(s['특징'])}\n"
                f"  ├ 피크타임: {s['시간']}\n"
                f"  ├ 추천메뉴: {', '.join(s['메뉴'])}\n"
                f"  └ 가격민감도: {s['민감도']}"
            )

        return f"🎯 {district} 타겟 전략\n{fmt(pri)}\n\n보조\n{fmt(sec)}"

# ========================
# 5. 리포트 & 실행 스크립트
# ========================

def run_analysis():
    """엔드-투-엔드 실행 함수"""

    # ▶︎ STEP 0: API 키 설정 (환경 변수 → 인자 전달)
    client_id = os.getenv("NAVER_CLIENT_ID", "QjzwUZBwBgZijbaadyKV")
    client_secret = os.getenv("NAVER_CLIENT_SECRET", "fIqgUfyng8")

    client = NaverAPIClient(client_id, client_secret)
    trend_analyzer = GopchangTrendAnalyzer(client)

    # ▶︎ STEP 1: 데이터 수집 & 전처리
    raw = trend_analyzer.collect(days=30)
    df = trend_analyzer.preprocess(raw)

    # ▶︎ STEP 2: 주요 상권 인사이트 & 타겟팅
    districts_info = [
        ("강남역", "프리미엄 비즈니스"),
        ("명동", "관광/전통상권"),
        ("홍대", "대학가"),
        ("신촌", "대학가"),
        ("종로", "관광/전통상권"),
    ]

    target_analyzer = GopchangTargetAnalyzer()

    insights: List[str] = []
    for d, d_type in districts_info:
        insights.append(trend_analyzer.district_insight(df, d))
        insights.append(target_analyzer.strategy(d, d_type))
        insights.append("-" * 40)

    # ▶︎ STEP 3: 워드클라우드 예시 (제목 기준)
    wc_text = " ".join(df["title"].tolist())
    WordCloud(
        font_path="/System/Library/Fonts/AppleGothic.ttf",
        background_color="white",
        width=800,
        height=400,
    ).generate(wc_text).to_file("wordcloud_gopchang.png")

    # ▶︎ STEP 4: 콘솔 리포트 출력 & 저장
    report = (
        "\n".join(insights)
        + "\n" + "=" * 60 + "\n"
        + f"Generated: {datetime.now():%Y-%m-%d %H:%M}"
    )
    print(report)
    with open("gopchang_report.txt", "w", encoding="utf-8") as fp:
        fp.write(report)

    print("✅ 분석 완료 — 'gopchang_report.txt', 'wordcloud_gopchang.png' 생성")


if __name__ == "__main__":
    run_analysis()


In [ ]:
pip install pandas requests

In [ ]:
# 🚀 바로 실행 가능한 네이버 API 곱창 분석 코드
# API 키만 입력하면 즉시 실행 가능!
# ================================================================================

import requests
import pandas as pd
import json
from datetime import datetime
import time

class NaverGopchangAnalyzer:
    """곱창집 창업을 위한 네이버 API 분석기"""
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = "https://openapi.naver.com/v1/search"
        self.headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
        }
        
    def _api_call(self, endpoint, query, display=100, sort='date'):
        """API 호출 공통 함수"""
        url = f"{self.base_url}/{endpoint}.json"
        params = {
            'query': query,
            'display': display,
            'sort': sort
        }
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                print(f"❌ API 호출 실패: {response.status_code}")
                return None
        except Exception as e:
            print(f"❌ 오류 발생: {e}")
            return None

    def analyze_district_popularity(self, districts=None):
        """🏆 지역별 곱창 인기도 분석 - 즉시 실행 가능!"""
        
        if districts is None:
            districts = ["강남역", "홍대", "신촌", "명동", "종로", "잠실", "건대", "신림"]
        
        print("🔍 지역별 곱창 인기도 분석 중...")
        print("=" * 50)
        
        results = []
        
        for district in districts:
            print(f"📍 {district} 분석 중...", end=" ")
            
            # 블로그 검색
            blog_data = self._api_call('blog', f"{district} 곱창", display=100)
            blog_count = blog_data['total'] if blog_data else 0
            
            # 뉴스 검색  
            news_data = self._api_call('news', f"{district} 곱창", display=50)
            news_count = news_data['total'] if news_data else 0
            
            # 카페 검색
            cafe_data = self._api_call('cafearticle', f"{district} 곱창", display=50)
            cafe_count = cafe_data['total'] if cafe_data else 0
            
            total_mentions = blog_count + news_count + cafe_count
            
            results.append({
                'district': district,
                'blog_mentions': blog_count,
                'news_mentions': news_count, 
                'cafe_mentions': cafe_count,
                'total_mentions': total_mentions
            })
            
            print(f"✅ 총 {total_mentions:,}건")
            time.sleep(0.1)  # API 호출 제한 고려
        
        # 결과 정렬 및 출력
        results.sort(key=lambda x: x['total_mentions'], reverse=True)
        
        print(f"\n🏆 지역별 곱창 언급량 순위:")
        print("-" * 60)
        for i, result in enumerate(results, 1):
            print(f"{i:2d}. {result['district']:<8} "
                  f"총 {result['total_mentions']:>6,}건 "
                  f"(블로그:{result['blog_mentions']:>4,} "
                  f"뉴스:{result['news_mentions']:>3,} "
                  f"카페:{result['cafe_mentions']:>3,})")
        
        return results

    def analyze_competition_vs_other_foods(self):
        """🍖 곱창 vs 다른 음식 경쟁력 분석"""
        
        foods = {
            "곱창": ["곱창", "막창"],
            "치킨": ["치킨", "닭갈비"], 
            "삼겹살": ["삼겹살", "고기구이"],
            "족발": ["족발", "보쌈"],
            "마라탕": ["마라탕", "마라샹궈"]
        }
        
        print("\n🍖 곱창 vs 다른 음식 경쟁력 분석")
        print("=" * 40)
        
        competition_results = {}
        
        for food_category, keywords in foods.items():
            total_mentions = 0
            
            for keyword in keywords:
                blog_data = self._api_call('blog', keyword, display=100)
                if blog_data:
                    total_mentions += blog_data['total']
                time.sleep(0.1)
            
            competition_results[food_category] = total_mentions
            print(f"📊 {food_category:<6}: {total_mentions:>8,}건")
        
        # 곱창의 상대적 위치 계산
        gopchang_rank = sorted(competition_results.items(), 
                              key=lambda x: x[1], reverse=True)
        
        print(f"\n💡 분석 결과:")
        for i, (food, count) in enumerate(gopchang_rank, 1):
            emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "📍"
            print(f"   {emoji} {i}위: {food} ({count:,}건)")
        
        return competition_results

    def analyze_time_demand_patterns(self):
        """⏰ 시간대별 곱창 수요 패턴 분석"""
        
        time_patterns = {
            "점심": "곱창 점심",
            "저녁": "곱창 저녁", 
            "야식": "곱창 야식",
            "새벽": "곱창 새벽",
            "주말": "곱창 주말",
            "평일": "곱창 평일",
            "금요일": "곱창 금요일"
        }
        
        print("\n⏰ 시간대별 곱창 수요 패턴 분석")
        print("=" * 35)
        
        time_results = {}
        
        for time_type, keyword in time_patterns.items():
            blog_data = self._api_call('blog', keyword, display=50)
            mentions = blog_data['total'] if blog_data else 0
            time_results[time_type] = mentions
            
            print(f"🕐 {time_type:<4}: {mentions:>5,}건")
            time.sleep(0.1)
        
        # 최고 수요 시간대 찾기
        peak_time = max(time_results.items(), key=lambda x: x[1])
        print(f"\n🔥 최고 수요: {peak_time[0]} ({peak_time[1]:,}건)")
        
        return time_results

    def analyze_price_sensitivity(self):
        """💰 곱창 가격 민감도 분석"""
        
        price_keywords = [
            "곱창 1만원", "곱창 2만원", "곱창 3만원", "곱창 4만원",
            "곱창 비싸다", "곱창 저렴", "곱창 가성비", "곱창 가격"
        ]
        
        print("\n💰 곱창 가격 민감도 분석")
        print("=" * 30)
        
        price_results = {}
        
        for keyword in price_keywords:
            blog_data = self._api_call('blog', keyword, display=30)
            mentions = blog_data['total'] if blog_data else 0
            price_results[keyword] = mentions
            
            print(f"💵 {keyword:<12}: {mentions:>4,}건")
            time.sleep(0.1)
        
        return price_results

    def analyze_customer_complaints(self, district="강남"):
        """😤 고객 불만사항 분석"""
        
        complaint_keywords = [
            f"{district} 곱창 별로",
            f"{district} 곱창 실망", 
            f"{district} 곱창 비싸",
            f"{district} 곱창 불친절",
            f"{district} 곱창 더럽"
        ]
        
        print(f"\n😤 {district} 지역 곱창집 고객 불만 분석")
        print("=" * 40)
        
        complaints = {}
        total_complaints = 0
        
        for keyword in complaint_keywords:
            blog_data = self._api_call('blog', keyword, display=20)
            count = blog_data['total'] if blog_data else 0
            complaints[keyword] = count
            total_complaints += count
            
            if count > 0:
                issue_type = keyword.split()[-1]
                print(f"⚠️  {issue_type:<6}: {count:>3,}건")
            
            time.sleep(0.1)
        
        if total_complaints == 0:
            print("✅ 특별한 불만사항이 발견되지 않았습니다!")
        else:
            print(f"\n📊 총 불만 언급: {total_complaints}건")
        
        return complaints

    def find_trending_keywords(self):
        """🔥 곱창 관련 트렌딩 키워드 발굴"""
        
        trending_concepts = [
            "곱창 다이어트", "곱창 건강", "곱창 ASMR",
            "곱창 먹방", "곱창 인스타", "곱창 MZ세대",
            "곱창 배달", "곱창 테이크아웃", "곱창 홈파티"
        ]
        
        print("\n🔥 곱창 트렌딩 키워드 분석")
        print("=" * 30)
        
        trend_scores = {}
        
        for keyword in trending_concepts:
            blog_data = self._api_call('blog', keyword, display=30)
            mentions = blog_data['total'] if blog_data else 0
            
            # 2024년 언급량으로 트렌드 점수 계산
            recent_data = self._api_call('blog', f"{keyword} 2024", display=10)
            recent_mentions = recent_data['total'] if recent_data else 0
            
            # 트렌드 점수 = 최근 언급 비율
            trend_score = (recent_mentions / max(mentions, 1)) * 100
            trend_scores[keyword] = {
                'total': mentions,
                'recent': recent_mentions,
                'trend_score': trend_score
            }
            
            trend_emoji = "🚀" if trend_score > 50 else "📈" if trend_score > 20 else "📊"
            print(f"{trend_emoji} {keyword:<12}: {mentions:>3,}건 (트렌드:{trend_score:>4.1f}%)")
            
            time.sleep(0.1)
        
        return trend_scores

    def generate_location_insight(self, target_district):
        """🎯 특정 지역 심층 분석 리포트"""
        
        print(f"\n🎯 {target_district} 곱창 시장 심층 분석")
        print("=" * 50)
        
        # 1. 전체 시장 규모
        market_data = self._api_call('blog', f"{target_district} 곱창", display=100)
        market_size = market_data['total'] if market_data else 0
        
        # 2. 경쟁업체 조사
        competitors = self._api_call('local', f"{target_district} 곱창집", display=5)
        competitor_count = len(competitors['items']) if competitors else 0
        
        # 3. 고객 만족도
        positive_data = self._api_call('blog', f"{target_district} 곱창 맛있다", display=30)
        positive_mentions = positive_data['total'] if positive_data else 0
        
        # 4. 가격 관련 언급
        price_data = self._api_call('blog', f"{target_district} 곱창 가격", display=30)
        price_mentions = price_data['total'] if price_data else 0
        
        # 결과 출력
        print(f"📊 시장 규모: {market_size:,}개 온라인 언급")
        print(f"🏪 경쟁업체: {competitor_count}개 곱창집 발견")
        print(f"😊 긍정 평가: {positive_mentions:,}건")
        print(f"💰 가격 논의: {price_mentions:,}건")
        
        # 시장 기회 점수 계산 (간단한 공식)
        opportunity_score = min(100, 
            (market_size * 0.4) + 
            (max(0, 10 - competitor_count) * 5) + 
            (positive_mentions * 0.3)
        )
        
        print(f"\n🎯 시장 기회 점수: {opportunity_score:.1f}/100점")
        
        if opportunity_score >= 70:
            print("✅ 매우 유망한 지역입니다!")
        elif opportunity_score >= 50:
            print("⚠️  신중한 검토가 필요한 지역입니다.")
        else:
            print("❌ 진입이 어려운 지역으로 보입니다.")
        
        return {
            'market_size': market_size,
            'competitors': competitor_count,
            'positive_mentions': positive_mentions,
            'opportunity_score': opportunity_score
        }

    def run_comprehensive_analysis(self, target_district="강남역"):
        """🚀 종합 분석 실행"""
        
        print("🍖 네이버 API 곱창집 창업 분석 시작!")
        print(f"📅 분석 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        
        # 전체 분석 실행
        try:
            # 1. 지역별 인기도
            regional_results = self.analyze_district_popularity()
            
            # 2. 음식 경쟁력
            food_competition = self.analyze_competition_vs_other_foods()
            
            # 3. 시간대별 수요
            time_patterns = self.analyze_time_demand_patterns()
            
            # 4. 가격 민감도
            price_analysis = self.analyze_price_sensitivity()
            
            # 5. 트렌딩 키워드
            trending = self.find_trending_keywords()
            
            # 6. 타겟 지역 심층 분석
            location_insight = self.generate_location_insight(target_district)
            
            # 7. 종합 리포트 생성
            self._generate_final_report(regional_results, food_competition, 
                                      time_patterns, location_insight, target_district)
            
        except Exception as e:
            print(f"❌ 분석 중 오류 발생: {e}")
            print("💡 API 키와 인터넷 연결을 확인해주세요.")

    def _generate_final_report(self, regional_results, food_competition, 
                              time_patterns, location_insight, target_district):
        """📋 최종 리포트 생성"""
        
        print(f"\n📋 {target_district} 곱창집 창업 분석 최종 리포트")
        print("=" * 50)
        
        # TOP 3 추천 지역
        top_regions = regional_results[:3]
        print(f"🏆 추천 창업 지역 TOP 3:")
        for i, region in enumerate(top_regions, 1):
            print(f"   {i}. {region['district']} - {region['total_mentions']:,}건")
        
        # 곱창 시장 포지션
        gopchang_rank = list(food_competition.keys()).index('곱창') + 1
        print(f"\n📊 곱창 시장 포지션: 음식업계 {gopchang_rank}위")
        
        # 최적 운영 시간
        peak_time = max(time_patterns.items(), key=lambda x: x[1])
        print(f"⏰ 최적 운영 시간: {peak_time[0]} (수요 {peak_time[1]:,}건)")
        
        # 타겟 지역 평가
        opportunity = location_insight['opportunity_score']
        print(f"🎯 {target_district} 창업 적합도: {opportunity:.1f}점")
        
        # 핵심 제안사항
        print(f"\n💡 핵심 제안사항:")
        if opportunity >= 70:
            print(f"   ✅ {target_district}는 곱창집 창업에 적합한 지역입니다")
            print(f"   ✅ {peak_time[0]} 시간대 특화 운영 권장")
        else:
            alternatives = [r['district'] for r in top_regions if r['district'] != target_district][:2]
            print(f"   ⚠️  {target_district} 대신 {', '.join(alternatives)} 지역 고려")
        
        print(f"   📈 온라인 마케팅 필수 (높은 온라인 관심도)")
        print(f"   🎪 차별화된 컨셉으로 경쟁력 확보 필요")

# 🚀 실행 함수
def main():
    """메인 실행 함수"""
    
    print("🔑 네이버 API 키를 입력해주세요:")
    print("(네이버 개발자센터에서 발급받으세요: https://developers.naver.com/)")
    
    # API 키 입력 (실제 사용 시 여기에 입력)
    CLIENT_ID = input("Client ID: ").strip()
    CLIENT_SECRET = input("Client Secret: ").strip()
    
    if not CLIENT_ID or not CLIENT_SECRET:
        print("❌ API 키가 입력되지 않았습니다.")
        print("\n💡 테스트를 원하시면 다음과 같이 직접 입력하세요:")
        print('analyzer = NaverGopchangAnalyzer("your_id", "your_secret")')
        print('analyzer.run_comprehensive_analysis("원하는지역")')
        return
    
    # 분석 실행
    target_area = input("분석할 지역을 입력하세요 (예: 강남역): ").strip() or "강남역"
    
    analyzer = NaverGopchangAnalyzer(CLIENT_ID, CLIENT_SECRET)
    analyzer.run_comprehensive_analysis(target_area)
    
    print(f"\n✅ 분석 완료! 결과를 참고하여 현명한 창업 결정하세요! 🍖")

# 개별 분석 함수들 (원하는 분석만 실행 가능)
def quick_popularity_check(client_id, client_secret):
    """빠른 인기도 체크 (1분)"""
    analyzer = NaverGopchangAnalyzer(client_id, client_secret)
    return analyzer.analyze_district_popularity()

def quick_competition_check(client_id, client_secret):
    """빠른 경쟁 현황 체크 (30초)"""
    analyzer = NaverGopchangAnalyzer(client_id, client_secret)
    return analyzer.analyze_competition_vs_other_foods()

def quick_location_check(client_id, client_secret, location):
    """특정 지역 빠른 체크 (30초)"""
    analyzer = NaverGopchangAnalyzer(client_id, client_secret)
    return analyzer.generate_location_insight(location)

if __name__ == "__main__":
    main()
    
    # 또는 개별 실행 예시:
    # analyzer = NaverGopchangAnalyzer("your_id", "your_secret")
    # analyzer.analyze_district_popularity()  # 지역별 인기도만
    # analyzer.analyze_time_demand_patterns()  # 시간대별 분석만
    # analyzer.generate_location_insight("홍대")  # 홍대 분석만

In [ ]:
# 🔍 홍대/잠실/종로 TOP 3 지역 심층 분석
# 각 지역별 상세 특성 및 창업 기회 분석
# ================================================================================

import requests
import time
from datetime import datetime

class DetailedRegionAnalyzer:
    """TOP 3 지역 심층 분석기"""
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
        }
        
    def search_api(self, search_type, query, display=50):
        """네이버 API 검색"""
        url = f"https://openapi.naver.com/v1/search/{search_type}.json"
        params = {'query': query, 'display': display, 'sort': 'date'}
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                return None
        except:
            return None

    def analyze_customer_characteristics(self, region):
        """🎯 지역별 고객 특성 분석"""
        
        customer_keywords = {
            "대학생": f"{region} 곱창 대학생",
            "직장인": f"{region} 곱창 회식", 
            "가족": f"{region} 곱창 가족",
            "커플": f"{region} 곱창 데이트",
            "관광객": f"{region} 곱창 관광",
            "외국인": f"{region} 곱창 외국인"
        }
        
        print(f"\n🎯 {region} 고객 특성 분석")
        print("=" * 30)
        
        customer_data = {}
        total_customer_mentions = 0
        
        for customer_type, keyword in customer_keywords.items():
            data = self.search_api('blog', keyword, display=30)
            mentions = data['total'] if data else 0
            customer_data[customer_type] = mentions
            total_customer_mentions += mentions
            
            if mentions > 0:
                percentage = (mentions / max(total_customer_mentions, 1)) * 100
                bar = "█" * min(15, mentions // 10)
                print(f"👥 {customer_type:<6}: {mentions:>4,}건 {bar}")
            
            time.sleep(0.2)
        
        # 주요 고객층 찾기
        if total_customer_mentions > 0:
            main_customers = sorted(customer_data.items(), key=lambda x: x[1], reverse=True)[:3]
            print(f"\n🎯 주요 고객층:")
            for i, (customer_type, count) in enumerate(main_customers, 1):
                if count > 0:
                    print(f"   {i}. {customer_type} ({count:,}건)")
        
        return customer_data

    def analyze_competition_intensity(self, region):
        """🏪 경쟁 강도 분석"""
        
        competition_keywords = [
            f"{region} 곱창집 많다",
            f"{region} 곱창 맛집", 
            f"{region} 곱창 추천",
            f"{region} 곱창집 경쟁",
            f"{region} 곱창 포화"
        ]
        
        print(f"\n🏪 {region} 경쟁 환경 분석")
        print("=" * 25)
        
        competition_scores = {}
        
        # 지역 내 곱창집 검색
        local_data = self.search_api('local', f"{region} 곱창집", display=20)
        if local_data and 'items' in local_data:
            restaurant_count = len(local_data['items'])
            print(f"🏪 등록된 곱창집: {restaurant_count}개")
            
            # 상위 곱창집들의 온라인 언급량
            if restaurant_count > 0:
                print(f"📊 주요 경쟁업체:")
                for i, restaurant in enumerate(local_data['items'][:5], 1):
                    name = restaurant.get('title', '').replace('<b>', '').replace('</b>', '')
                    # 업체명으로 검색 (간단히)
                    mentions = self.search_api('blog', name, display=10)
                    mention_count = mentions['total'] if mentions else 0
                    print(f"   {i}. {name[:15]}: {mention_count:,}건")
                    time.sleep(0.2)
        else:
            restaurant_count = 0
            print(f"🏪 등록된 곱창집: 검색되지 않음")
        
        # 경쟁 관련 키워드 분석
        for keyword in competition_keywords:
            data = self.search_api('blog', keyword, display=20)
            mentions = data['total'] if data else 0
            competition_scores[keyword] = mentions
            time.sleep(0.2)
        
        # 경쟁 강도 점수 계산 (0-100)
        competition_intensity = min(100, restaurant_count * 5)
        
        print(f"\n📊 경쟁 강도: {competition_intensity}점/100점")
        if competition_intensity >= 70:
            print("⚠️ 경쟁 매우 치열")
        elif competition_intensity >= 40:
            print("📊 적절한 경쟁")
        else:
            print("✅ 경쟁 여유로움")
            
        return {
            'restaurant_count': restaurant_count,
            'competition_intensity': competition_intensity,
            'competition_scores': competition_scores
        }

    def analyze_price_trends(self, region):
        """💰 지역별 가격 트렌드 분석"""
        
        price_keywords = {
            "저가": f"{region} 곱창 저렴",
            "중가": f"{region} 곱창 2만원",
            "고가": f"{region} 곱창 비싸다",
            "가성비": f"{region} 곱창 가성비"
        }
        
        print(f"\n💰 {region} 가격 트렌드 분석")
        print("=" * 25)
        
        price_data = {}
        total_price_mentions = 0
        
        for price_type, keyword in price_keywords.items():
            data = self.search_api('blog', keyword, display=30)
            mentions = data['total'] if data else 0
            price_data[price_type] = mentions
            total_price_mentions += mentions
            
            if mentions > 0:
                percentage = (mentions / max(total_price_mentions, 1)) * 100
                print(f"💵 {price_type:<4}: {mentions:>4,}건 ({percentage:4.1f}%)")
            
            time.sleep(0.2)
        
        # 가격 포지셔닝 추천
        if total_price_mentions > 0:
            sorted_prices = sorted(price_data.items(), key=lambda x: x[1], reverse=True)
            dominant_price = sorted_prices[0]
            
            print(f"\n🎯 주요 가격대: {dominant_price[0]} ({dominant_price[1]:,}건)")
            
            if dominant_price[0] == "가성비":
                recommendation = "가성비 중심 전략 권장"
            elif dominant_price[0] == "저가":
                recommendation = "저가격 경쟁 주의 필요"
            elif dominant_price[0] == "고가":
                recommendation = "프리미엄 시장 기회"
            else:
                recommendation = "중간 가격대 안정적"
                
            print(f"💡 추천 전략: {recommendation}")
        
        return price_data

    def analyze_unique_opportunities(self, region):
        """🔍 지역별 고유 기회 분석"""
        
        # 지역별 특화 키워드
        if "홍대" in region:
            opportunity_keywords = [
                f"{region} 곱창 클럽",
                f"{region} 곱창 펜트하우스",
                f"{region} 곱창 힙스터",
                f"{region} 곱창 인스타",
                f"{region} 곱창 젊은",
                f"{region} 곱창 늦은시간"
            ]
        elif "잠실" in region:
            opportunity_keywords = [
                f"{region} 곱창 가족",
                f"{region} 곱창 롯데월드",
                f"{region} 곱창 석촌호수",
                f"{region} 곱창 아이",
                f"{region} 곱창 주말",
                f"{region} 곱창 넓은"
            ]
        elif "종로" in region:
            opportunity_keywords = [
                f"{region} 곱창 전통",
                f"{region} 곱창 관광",
                f"{region} 곱창 외국인",
                f"{region} 곱창 한국",
                f"{region} 곱창 역사",
                f"{region} 곱창 문화"
            ]
        else:
            opportunity_keywords = [f"{region} 곱창 특별"]
        
        print(f"\n🔍 {region} 고유 기회 분석")
        print("=" * 25)
        
        opportunities = {}
        
        for keyword in opportunity_keywords:
            data = self.search_api('blog', keyword, display=20)
            mentions = data['total'] if data else 0
            if mentions > 0:
                opportunities[keyword] = mentions
                clean_keyword = keyword.replace(f"{region} 곱창 ", "")
                print(f"💡 {clean_keyword:<8}: {mentions:>3,}건")
            time.sleep(0.2)
        
        return opportunities

    def analyze_time_patterns(self, region):
        """⏰ 지역별 시간대 패턴 분석"""
        
        time_keywords = {
            "점심": f"{region} 곱창 점심",
            "저녁": f"{region} 곱창 저녁", 
            "야식": f"{region} 곱창 야식",
            "새벽": f"{region} 곱창 새벽",
            "주말": f"{region} 곱창 주말"
        }
        
        print(f"\n⏰ {region} 시간대별 수요")
        print("=" * 20)
        
        time_data = {}
        
        for time_type, keyword in time_keywords.items():
            data = self.search_api('blog', keyword, display=30)
            mentions = data['total'] if data else 0
            time_data[time_type] = mentions
            
            if mentions > 0:
                bar = "█" * min(15, mentions // 50)
                print(f"🕐 {time_type:<4}: {mentions:>4,}건 {bar}")
            
            time.sleep(0.2)
        
        # 최적 운영시간 제안
        peak_time = max(time_data.items(), key=lambda x: x[1])
        print(f"\n🔥 피크시간: {peak_time[0]} ({peak_time[1]:,}건)")
        
        return time_data

    def generate_region_swot(self, region, customer_data, competition_data, price_data, opportunities, time_data):
        """📋 지역별 SWOT 분석"""
        
        print(f"\n📋 {region} SWOT 분석")
        print("=" * 30)
        
        # Strengths (강점)
        print("💪 Strengths (강점):")
        
        # 고객 다양성
        active_customers = [k for k, v in customer_data.items() if v > 100]
        if len(active_customers) >= 3:
            print("   ✅ 다양한 고객층 확보")
        
        # 경쟁 적절성
        if competition_data['competition_intensity'] < 70:
            print("   ✅ 적절한 경쟁 환경")
        
        # 고유 기회
        if opportunities:
            top_opportunity = max(opportunities.items(), key=lambda x: x[1])
            clean_opp = top_opportunity[0].replace(f"{region} 곱창 ", "")
            print(f"   ✅ {clean_opp} 특화 기회")
        
        # Weaknesses (약점)
        print("\n⚠️ Weaknesses (약점):")
        
        if competition_data['competition_intensity'] >= 70:
            print("   ❌ 높은 경쟁 강도")
        
        if competition_data['restaurant_count'] >= 15:
            print("   ❌ 많은 기존 업체")
        
        # Opportunities (기회)
        print("\n🌟 Opportunities (기회):")
        
        # 가격 기회
        if price_data.get('가성비', 0) > price_data.get('고가', 0):
            print("   💡 가성비 시장 공략")
        
        # 시간대 기회
        peak_time = max(time_data.items(), key=lambda x: x[1])
        print(f"   💡 {peak_time[0]} 시간대 특화")
        
        # 고유 기회
        for keyword, mentions in sorted(opportunities.items(), key=lambda x: x[1], reverse=True)[:2]:
            clean_keyword = keyword.replace(f"{region} 곱창 ", "")
            print(f"   💡 {clean_keyword} 컨셉 개발")
        
        # Threats (위협)
        print("\n🚨 Threats (위협):")
        
        if competition_data['competition_intensity'] >= 50:
            print("   ⚡ 경쟁 심화")
        
        print("   ⚡ 임대료 상승 가능성")
        print("   ⚡ 고객 취향 변화")

    def comprehensive_region_analysis(self, region):
        """🔍 지역 종합 분석"""
        
        print(f"\n🔍 {region} 종합 심층 분석")
        print("=" * 50)
        
        # 1. 고객 특성 분석
        customer_data = self.analyze_customer_characteristics(region)
        
        # 2. 경쟁 환경 분석
        competition_data = self.analyze_competition_intensity(region)
        
        # 3. 가격 트렌드 분석
        price_data = self.analyze_price_trends(region)
        
        # 4. 고유 기회 분석
        opportunities = self.analyze_unique_opportunities(region)
        
        # 5. 시간대 패턴 분석
        time_data = self.analyze_time_patterns(region)
        
        # 6. SWOT 분석
        self.generate_region_swot(region, customer_data, competition_data, price_data, opportunities, time_data)
        
        # 7. 종합 점수 계산
        total_score = self.calculate_comprehensive_score(customer_data, competition_data, price_data, opportunities)
        
        print(f"\n🎯 {region} 종합 점수: {total_score:.1f}/100점")
        
        return {
            'region': region,
            'customer_data': customer_data,
            'competition_data': competition_data,
            'price_data': price_data,
            'opportunities': opportunities,
            'time_data': time_data,
            'total_score': total_score
        }
    
    def calculate_comprehensive_score(self, customer_data, competition_data, price_data, opportunities):
        """종합 점수 계산"""
        
        # 고객 다양성 점수 (0-30점)
        active_customers = len([v for v in customer_data.values() if v > 50])
        customer_score = min(30, active_customers * 5)
        
        # 경쟁 환경 점수 (0-25점) - 경쟁이 적을수록 높은 점수
        competition_score = max(0, 25 - (competition_data['competition_intensity'] * 0.25))
        
        # 가격 기회 점수 (0-20점)
        price_score = min(20, sum(price_data.values()) * 0.001)
        
        # 고유 기회 점수 (0-25점)
        opportunity_score = min(25, len(opportunities) * 5)
        
        total_score = customer_score + competition_score + price_score + opportunity_score
        
        return total_score

    def compare_top3_regions(self):
        """🏆 TOP 3 지역 비교 분석"""
        
        regions = ["홍대", "잠실", "종로"]
        
        print("🏆 TOP 3 지역 심층 비교 분석")
        print("=" * 60)
        print(f"📅 분석 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        region_results = {}
        
        for region in regions:
            print(f"\n{'='*60}")
            print(f"📍 {region} 분석 시작")
            print(f"{'='*60}")
            
            result = self.comprehensive_region_analysis(region)
            region_results[region] = result
            
            print(f"\n✅ {region} 분석 완료!")
            time.sleep(1)  # API 부하 방지
        
        # 최종 비교 및 추천
        self.generate_final_comparison(region_results)
        
        return region_results
    
    def generate_final_comparison(self, region_results):
        """📊 최종 비교 및 추천"""
        
        print(f"\n📊 TOP 3 지역 최종 비교")
        print("=" * 40)
        
        # 종합 점수 순위
        sorted_regions = sorted(region_results.items(), key=lambda x: x[1]['total_score'], reverse=True)
        
        print(f"🏆 종합 점수 순위:")
        for i, (region, data) in enumerate(sorted_regions, 1):
            medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉"
            print(f"   {medal} {i}위: {region} ({data['total_score']:.1f}점)")
        
        # 카테고리별 최고
        print(f"\n🎯 카테고리별 최고 지역:")
        
        # 고객 다양성
        customer_winner = max(region_results.items(), 
                            key=lambda x: len([v for v in x[1]['customer_data'].values() if v > 50]))
        print(f"   👥 고객 다양성: {customer_winner[0]}")
        
        # 경쟁 여유도
        competition_winner = min(region_results.items(),
                               key=lambda x: x[1]['competition_data']['competition_intensity'])
        print(f"   🏪 경쟁 여유도: {competition_winner[0]}")
        
        # 고유 기회
        opportunity_winner = max(region_results.items(),
                               key=lambda x: len(x[1]['opportunities']))
        print(f"   💡 고유 기회: {opportunity_winner[0]}")
        
        # 최종 추천
        winner = sorted_regions[0]
        print(f"\n🎯 최종 추천: {winner[0]}")
        print(f"📊 추천 이유:")
        
        winner_data = winner[1]
        
        # 강점 요약
        active_customers = len([v for v in winner_data['customer_data'].values() if v > 50])
        print(f"   ✅ {active_customers}개 고객층 확보")
        print(f"   ✅ 경쟁강도 {winner_data['competition_data']['competition_intensity']}점")
        print(f"   ✅ {len(winner_data['opportunities'])}개 고유 기회")
        
        # 구체적 전략 제안
        print(f"\n💡 {winner[0]} 창업 전략:")
        
        # 주요 고객층
        main_customers = sorted(winner_data['customer_data'].items(), key=lambda x: x[1], reverse=True)[:2]
        if main_customers[0][1] > 0:
            print(f"   🎯 주요 타겟: {main_customers[0][0]}")
        
        # 최적 시간대
        peak_time = max(winner_data['time_data'].items(), key=lambda x: x[1])
        print(f"   ⏰ 최적 운영: {peak_time[0]} 특화")
        
        # 가격 전략
        main_price = max(winner_data['price_data'].items(), key=lambda x: x[1])
        print(f"   💰 가격 전략: {main_price[0]} 중심")
        
        # 차별화 포인트
        if winner_data['opportunities']:
            top_opp = max(winner_data['opportunities'].items(), key=lambda x: x[1])
            clean_opp = top_opp[0].replace(f"{winner[0]} 곱창 ", "")
            print(f"   🔥 차별화: {clean_opp} 컨셉")

# 🚀 실행 함수
def analyze_top3_regions(client_id, client_secret):
    """TOP 3 지역 분석 실행"""
    
    analyzer = DetailedRegionAnalyzer(client_id, client_secret)
    results = analyzer.compare_top3_regions()
    
    return results

# 개별 지역 분석 함수
def analyze_single_region(client_id, client_secret, region):
    """특정 지역만 분석"""
    
    analyzer = DetailedRegionAnalyzer(client_id, client_secret)
    result = analyzer.comprehensive_region_analysis(region)
    
    return result

if __name__ == "__main__":
    print("🔍 TOP 3 지역 심층 분석 도구")
    print("홍대, 잠실, 종로를 상세 비교합니다.")
    print("=" * 40)
    
    print("💡 사용법:")
    print("analyzer = DetailedRegionAnalyzer('your_id', 'your_secret')")
    print("results = analyzer.compare_top3_regions()")

In [ ]:
# 기존 API 키로 TOP 3 지역 심층 분석 실행
analyzer = DetailedRegionAnalyzer("QjzwUZBwBgZijbaadyKV", "fIqgUfyng8")
results = analyzer.compare_top3_regions()

In [ ]:
import requests
import time
from datetime import datetime

class DetailedRegionAnalyzer:
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret
        }
        
    def search_api(self, search_type, query, display=50):
        url = f"https://openapi.naver.com/v1/search/{search_type}.json"
        params = {'query': query, 'display': display, 'sort': 'date'}
        
        try:
            response = requests.get(url, headers=self.headers, params=params)
            if response.status_code == 200:
                return response.json()
            else:
                return None
        except:
            return None

# 실행
analyzer = DetailedRegionAnalyzer("QjzwUZBwBgZijbaadyKV", "fIqgUfyng8")

print("🔍 홍대, 잠실, 종로 TOP 3 지역 심층 분석 시작!")
print("=" * 60)

regions = ["홍대", "잠실", "종로"]
detailed_results = {}

for region in regions:
    print(f"\n📍 {region} 상세 분석 중...")
    
    # 1. 고객 특성 분석
    print(f"🎯 {region} 고객 특성:")
    customer_keywords = {
        "대학생": f"{region} 곱창 대학생",
        "직장인": f"{region} 곱창 회식", 
        "가족": f"{region} 곱창 가족",
        "커플": f"{region} 곱창 데이트",
        "관광객": f"{region} 곱창 관광"
    }
    
    customer_data = {}
    for customer_type, keyword in customer_keywords.items():
        data = analyzer.search_api('blog', keyword, display=30)
        mentions = data['total'] if data else 0
        customer_data[customer_type] = mentions
        if mentions > 0:
            print(f"   👥 {customer_type}: {mentions:,}건")
        time.sleep(0.3)
    
    # 2. 경쟁 환경 분석
    print(f"\n🏪 {region} 경쟁 환경:")
    local_data = analyzer.search_api('local', f"{region} 곱창집", display=10)
    restaurant_count = len(local_data['items']) if local_data and 'items' in local_data else 0
    print(f"   🏪 등록된 곱창집: {restaurant_count}개")
    
    # 3. 가격 민감도
    print(f"\n💰 {region} 가격 반응:")
    price_keywords = {
        "저렴": f"{region} 곱창 저렴",
        "가성비": f"{region} 곱창 가성비",
        "비싸다": f"{region} 곱창 비싸다"
    }
    
    price_data = {}
    for price_type, keyword in price_keywords.items():
        data = analyzer.search_api('blog', keyword, display=20)
        mentions = data['total'] if data else 0
        price_data[price_type] = mentions
        if mentions > 0:
            print(f"   💵 {price_type}: {mentions:,}건")
        time.sleep(0.3)
    
    # 4. 지역별 특화 기회
    print(f"\n🔍 {region} 특화 기회:")
    if "홍대" in region:
        special_keywords = [f"{region} 곱창 힙스터", f"{region} 곱창 클럽", f"{region} 곱창 젊은"]
    elif "잠실" in region:
        special_keywords = [f"{region} 곱창 가족", f"{region} 곱창 롯데월드", f"{region} 곱창 주말"]
    elif "종로" in region:
        special_keywords = [f"{region} 곱창 전통", f"{region} 곱창 관광", f"{region} 곱창 외국인"]
    
    opportunities = {}
    for keyword in special_keywords:
        data = analyzer.search_api('blog', keyword, display=20)
        mentions = data['total'] if data else 0
        if mentions > 0:
            clean_keyword = keyword.replace(f"{region} 곱창 ", "")
            opportunities[clean_keyword] = mentions
            print(f"   💡 {clean_keyword}: {mentions:,}건")
        time.sleep(0.3)
    
    # 5. 시간대별 수요
    print(f"\n⏰ {region} 시간대별 수요:")
    time_keywords = ["점심", "저녁", "야식", "주말"]
    time_data = {}
    
    for time_type in time_keywords:
        data = analyzer.search_api('blog', f"{region} 곱창 {time_type}", display=20)
        mentions = data['total'] if data else 0
        time_data[time_type] = mentions
        if mentions > 0:
            print(f"   🕐 {time_type}: {mentions:,}건")
        time.sleep(0.3)
    
    # 결과 저장
    detailed_results[region] = {
        'customer_data': customer_data,
        'restaurant_count': restaurant_count,
        'price_data': price_data,
        'opportunities': opportunities,
        'time_data': time_data
    }
    
    print(f"✅ {region} 분석 완료!\n")

# 최종 비교 분석
print("📊 TOP 3 지역 종합 비교")
print("=" * 50)

for region, data in detailed_results.items():
    print(f"\n🏆 {region} 종합 평가:")
    
    # 주요 고객층
    main_customer = max(data['customer_data'].items(), key=lambda x: x[1])
    if main_customer[1] > 0:
        print(f"   🎯 주요 고객: {main_customer[0]} ({main_customer[1]:,}건)")
    
    # 경쟁 강도
    competition_level = "높음" if data['restaurant_count'] >= 10 else "중간" if data['restaurant_count'] >= 5 else "낮음"
    print(f"   🏪 경쟁 강도: {competition_level} ({data['restaurant_count']}개)")
    
    # 가격 특성
    main_price = max(data['price_data'].items(), key=lambda x: x[1])
    if main_price[1] > 0:
        print(f"   💰 가격 특성: {main_price[0]} 중시 ({main_price[1]:,}건)")
    
    # 특화 기회
    if data['opportunities']:
        top_opportunity = max(data['opportunities'].items(), key=lambda x: x[1])
        print(f"   💡 특화 기회: {top_opportunity[0]} ({top_opportunity[1]:,}건)")
    
    # 최적 시간
    peak_time = max(data['time_data'].items(), key=lambda x: x[1])
    if peak_time[1] > 0:
        print(f"   ⏰ 피크 시간: {peak_time[0]} ({peak_time[1]:,}건)")